In [29]:
from pyspark.sql import SparkSession
import json
import pandas as pd
import requests
import concurrent.futures
import time
from pyspark.sql.functions import col
import msal


StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 31, Finished, Available, Finished)

In [ ]:

# Azure AD Credentials (Replace with your values)
TENANT_ID = ""
CLIENT_ID = ""
CLIENT_SECRET = ""

# Azure AD Authority & Scope
# AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
# SCOPE = ["https://api.fabric.microsoft.com/.default"]  # Fabric API Scope
# TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPE = ["https://analysis.windows.net/powerbi/api/.default"]  # Power BI Scpoe
TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/token"



# Global variables for token caching
ACCESS_TOKEN = None
EXPIRATION_TIME = 0  # Store expiration time

def get_access_token():
    """Fetch and cache an access token from Azure AD for Power BI API."""
    global ACCESS_TOKEN, EXPIRATION_TIME

    # Check if token is still valid
    if ACCESS_TOKEN and time.time() < EXPIRATION_TIME:
        return ACCESS_TOKEN  # Return cached token if still valid

    # Create MSAL client application
    app = msal.ConfidentialClientApplication(CLIENT_ID, authority=AUTHORITY, client_credential=CLIENT_SECRET)

    # Acquire token
    token_response = app.acquire_token_for_client(scopes=SCOPE)

    if "access_token" in token_response:
        ACCESS_TOKEN = token_response["access_token"]
        EXPIRATION_TIME = time.time() + token_response.get("expires_in", 3600) - 300  # Refresh 5 minutes before expiry
        return ACCESS_TOKEN
    else:
        raise Exception(f"Failed to get token: {token_response}")




StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 32, Finished, Available, Finished)

In [31]:
#Reading the Datafrom the delta file
workspacess_df = (
    spark.read.format("delta")
    .load("abfss://d3120490-76ae-4ef4-a440-2bd65732ccdc@onelake.dfs.fabric.microsoft.com/fecab367-5d3a-41c1-8037-7801192932ba/Tables/fabric_workpsaces")
)

StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 33, Finished, Available, Finished)

In [32]:
#Filter Workspaces for a capactity
filtered_workspaces_df = workspacess_df.filter(col("capacityId") == "76F4499E-05FF-44A9-8C2F-323EE14EA1A7")
workspaceIds_list = [row["id"] for row in filtered_workspaces_df.select("id").collect()]

StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 34, Finished, Available, Finished)

In [33]:
#Reading the Datafrom the delta file
datasets_df = (
    spark.read.format("delta")
    .load("abfss://d3120490-76ae-4ef4-a440-2bd65732ccdc@onelake.dfs.fabric.microsoft.com/fecab367-5d3a-41c1-8037-7801192932ba/Tables/datasets")
)

StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 35, Finished, Available, Finished)

In [34]:
#Filter Datsets for a capactity
filtered_datasets_df = datasets_df.filter(col("workspaceId").isin(workspaceIds_list))
datasetIds_list = [row["id"] for row in filtered_datasets_df.select("id").collect()]

StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 36, Finished, Available, Finished)

In [35]:
import requests
import pandas as pd
import json

def get_dataset_datasource_details(workspaceId, datasetId):
    """Fetch dataset datasource details from Power BI API and return as a DataFrame with all data sources."""
    token = get_access_token()
    headers = {"Authorization": f"Bearer {token}"}

    url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/datasources"
    response = requests.get(url, headers=headers)
    status_code = response.status_code

    records = []

    if status_code == 200:
        try:
            source_data = response.json().get("value", [])
            for details in source_data:
                connection = details.get("connectionDetails", {})

                # Extract server/path/db name
                if "path" in connection and "kind" in connection:
                    server_path = connection.get("path")
                    db_kind = connection.get("kind")
                elif "server" in connection and "database" in connection:
                    server_path = connection.get("server")
                    db_kind = connection.get("database")
                elif "url" in connection:
                    server_path = connection.get("url")
                    db_kind = "NA"
                else:
                    server_path = str(connection)
                    db_kind = "NA"

                record = {
                    "WorkspaceID": workspaceId,
                    "DatasetID": datasetId,
                    "DatasourceType": details.get("datasourceType", "NA"),
                    "Server_Path": server_path,
                    "Database_Kind": db_kind,
                    "DatasourceID": details.get("datasourceId", "NA"),
                    "GatewayID": details.get("gatewayId", "NA"),
                    "StatusCode": status_code
                }
                records.append(record)

            # If the list is empty, still add a row for tracking
            if not source_data:
                records.append({
                    "WorkspaceID": workspaceId,
                    "DatasetID": datasetId,
                    "DatasourceType": "NA",
                    "Server_Path": "NA",
                    "Database_Kind": "NA",
                    "DatasourceID": "NA",
                    "GatewayID": "NA",
                    "StatusCode": status_code
                })

        except Exception as e:
            records.append({
                "WorkspaceID": workspaceId,
                "DatasetID": datasetId,
                "DatasourceType": "NA",
                "Server_Path": "NA",
                "Database_Kind": "NA",
                "DatasourceID": "NA",
                "GatewayID": "NA",
                "StatusCode": status_code
            })
    else:
        records.append({
            "WorkspaceID": workspaceId,
            "DatasetID": datasetId,
            "DatasourceType": "NA",
            "Server_Path": "NA",
            "Database_Kind": "NA",
            "DatasourceID": "NA",
            "GatewayID": "NA",
            "StatusCode": status_code
        })

    return pd.DataFrame(records)


StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 37, Finished, Available, Finished)

In [38]:
all_temp_dfs = pd.DataFrame()  # List to store non-empty DataFrames

for row in filtered_datasets_df.toLocalIterator():  # No need for unpacking

    temp_df = get_dataset_datasource_details(row["workspaceId"], row["id"])

    all_temp_dfs = pd.concat([all_temp_dfs, temp_df], ignore_index=True)

StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 40, Finished, Available, Finished)

In [39]:
spark_dataset_source_details_df = spark.createDataFrame(all_temp_dfs)

StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 41, Finished, Available, Finished)

In [40]:
#Reading the Datafrom the delta file
existing_datasets_source_details = (
    spark.read.format("delta")
    .load("abfss://d3120490-76ae-4ef4-a440-2bd65732ccdc@onelake.dfs.fabric.microsoft.com/fecab367-5d3a-41c1-8037-7801192932ba/Tables/datasets_source_details")
)

StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 42, Finished, Available, Finished)

In [41]:
# First, get only the new DatasetIDs
new_records_df = spark_dataset_source_details_df.join(
    existing_datasets_source_details,
    on="DatasetID",
    how="left_anti"
)

# Append new records to existing dataset
updated_datasets_source_details = existing_datasets_source_details.unionByName(new_records_df)


StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 43, Finished, Available, Finished)

In [42]:
updated_datasets_source_details.write.mode("overwrite").saveAsTable("datasets_source_details")

StatementMeta(, 9a49198d-9ad9-4fcd-a7a1-35369c3badbc, 44, Finished, Available, Finished)